<a href="https://colab.research.google.com/github/CannedBread35/GitHub_TagAndRelease_Practice/blob/main/fine_tuned_YOLOv8_model_chess_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The below cell's output: determination if a GPU is available for training.

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Mar  5 01:25:24 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!nvidia-smi

Sun Mar  3 22:16:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

The below cell shows the amount of free space left on a file system. Space can have the following values: Space Used. Total amount of space allocated to existing files in the file system.

In [ ]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         108G   28G   81G  26% /
tmpfs            64M     0   64M   0% /dev
shm             5.8G  4.0K  5.8G   1% /dev/shm
/dev/root       2.0G  1.1G  849M  57% /usr/sbin/docker-init
tmpfs           6.4G  1.2M  6.4G   1% /var/colab
/dev/sda1        44G   29G   15G  66% /kaggle/input
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware
drive            15G  4.3G   11G  29% /content/drive


Mount Google Drive in Colab IDE for seamless I/O operations.

Purpose:
This code mounts your Google Drive into the Colab environment, enabling easy
and efficient Input/Output (I/O) operations between your Google Drive and the
Colab notebook.

How it works:
1. The code imports the necessary module "drive" from the "google.colab"
   library.
2. It then uses the "mount" function to establish a connection between your
   Google Drive and the "/content/drive" directory within the Colab
   environment.
3. The parameter "force_remount=True" ensures that the drive is remounted,
   allowing for any changes in your Google Drive to be reflected in the Colab
   notebook.

Instructions:
- After running this code cell, you will be prompted to authorize access to
  your Google Drive.
- Follow the provided link, grant the necessary permissions, and copy the
  authorization code.
- Paste the code into the designated input area in the notebook.
- Once mounted, you can navigate to "/content/drive" to access your Google
  Drive files.

Note:
- This is a one-time setup unless you restart the Colab runtime.
- Mounting Google Drive is useful for loading datasets, saving model
  checkpoints, and performing I/O operations with files stored in your Google
  Drive.

<!-- ################################################################################
# Mount Google Drive in Colab IDE for seamless I/O operations.

# Purpose:
# This code mounts your Google Drive into the Colab environment, enabling easy
# and efficient Input/Output (I/O) operations between your Google Drive and the
# Colab notebook.

# How it works:
# 1. The code imports the necessary module "drive" from the "google.colab"
#    library.
# 2. It then uses the "mount" function to establish a connection between your
#    Google Drive and the "/content/drive" directory within the Colab
#    environment.
# 3. The parameter "force_remount=True" ensures that the drive is remounted,
#    allowing for any changes in your Google Drive to be reflected in the Colab
#    notebook.

# Instructions:
# - After running this code cell, you will be prompted to authorize access to
#   your Google Drive.
# - Follow the provided link, grant the necessary permissions, and copy the
#   authorization code.
# - Paste the code into the designated input area in the notebook.
# - Once mounted, you can navigate to "/content/drive" to access your Google
#   Drive files.

# Note:
# - This is a one-time setup unless you restart the Colab runtime.
# - Mounting Google Drive is useful for loading datasets, saving model
#   checkpoints, and performing I/O operations with files stored in your Google
#   Drive.
################################################################################ -->

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)


Mounted at /content/drive


Install deep learning library named Ultralytics and import a specific class to
assist with computer vision tasks.

Purpose:
This code installs a deep learning library specifically for provision of tools
and utilities for computer vision tasks. A class, "YOLO", is imported tailored
specifically for the purpose of object detection. We are to customize and
adapt an object detection model to suit out project's specific requirements
and scenarios

Import YOLO class specifically tailored for object recognition tasks. In this
project, we will specifically use YOLO for object detection.

Note:
- While this project's unique requirements require YOLO for *object detection,
  YOLO class may also be used for other computer vision tasks such as the
  following: object segmentation

In [ ]:
!pip install ultralytics

from ultralytics import YOLO
from IPython.display import display, Image

# Pre-stratified sampling

OUTPUT OF THIS CELL: a numpy array that is a binary matrix that gets fed into
the stratification algorithm I am choosing to implement to optimize data
splitting.
I am extracting the labels from the folder of raw .xml files (where .xml files
are literally the annotation files for each image).
Cell output: a numpy array (binary matrix) where each index (of the numpy
array) is a type of chess piece of the board itself.
The value of the index is either a 0 or a 1, indicating if the type of chess
piece is there.

The following cell is practice, and correctly parses only one .xml file. Need
to parse an entire folder of thousands of .xml files.

In [ ]:
import xml.etree.ElementTree as ET
import numpy as np

# Assuming the XML content is stored in `annotation_path`
annotation_path = '/content/drive/MyDrive/images_gray_white_strain/annotations/Annotations/WIN_20240226_19_21_20_Pro.xml'

# Updated class mappings
# class_mappings = {
#     'board': 0,
#     'b-bishop': 1,
#     'b-king': 2,
#     'b-knight': 3,
#     'b-pawn': 4,
#     'b-queen': 5,
#     'b-rook': 6
# }
# Define the class_mapping dictionary
class_mappings = {
    'b-bishop': 0,
    'b-king': 1,
    'b-knight': 2,
    'b-pawn': 3,
    'b-queen': 4,
    'b-rook': 5,
    'board': 6,
    'w-bishop': 7,
    'w-king': 8,
    'w-knight': 9,
    'w-pawn': 10,
    'w-queen': 11,
    'w-rook': 12
}

def extract_labels_from_xml(xml_file, class_mappings):
    # Parse the XML file
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Initialize an array to represent the labels for the image
    image_labels = [0] * len(class_mappings)  # Initialize based on number of classes

    # Iterate through each object in the XML
    for obj in root.findall('.//object'):
        label = obj.find('name').text
        # Update the label array if the label is in class_mappings
        if label in class_mappings:
            image_labels[class_mappings[label]] = 1

    return image_labels

# Extract labels for the single image using the updated class mappings
labels_for_image = extract_labels_from_xml(annotation_path, class_mappings)

# Convert the labels into a numpy array
labels = np.array([labels_for_image])  # Making it 2D to match expected shape

print(labels)




[[1 1 1 1 1 1 1 1 1 1 1 1 1]]


Run this code before executing cell code blocks of performing stratified sampling

An example output of a binary matrix representing a single .xml annotation files:

[[1 0 0 1 0 0 0]]

Where the respective index mappings are as follows:

class_mappings = {
    'board': 0,
    'b-bishop': 1,
    'b-king': 2,
    'b-knight': 3,
    'b-pawn': 4,
    'b-queen': 5,
    'b-rook': 6
}

As you can see, the value of '1' in the binary matrix array at index 0 is indicative that the class 'board' is present in the .xml annotation file.
Furthermore, the value of '0' at index 1 in the binary matrix array is indicative that the class 'b-bishop' is present in the .xml annotation file as well. And so on.

The purpose of this cell is to have a large binary matrix representing the folder of .xml annotation files so that we can feed this binary matrix as an argument to a stratification algorithm.

The below cell, immediately down, is modified to handle creating the binary matrix representing ALL of the .xml files. Had to take baby steps to work up to processing an entire folder of thousands of .xml annotation files.


The below code moves the contents of one folder to another folder--order is preserved.

In [ ]:
import os
import shutil

# Path to the original annotations folder
original_folder = "/content/drive/MyDrive/Stratified/Stratified_White_Dataset/Validation/annotations"

# Path to the annotations_pt2 folder
pt2_folder = "/content/drive/MyDrive/Stratified/Stratified_White_Dataset/Validation"

# Get the list of files in annotations_pt2 folder
files_to_move = os.listdir(pt2_folder)

# Iterate through each file and move it to the original folder if it has a .xml extension
for file_name in files_to_move:
    if file_name.endswith(".xml"):
        src = os.path.join(pt2_folder, file_name)
        dst = os.path.join(original_folder, file_name)
        shutil.move(src, dst)

In [ ]:
# The following cell with code produces the output of a numpy array where each
# index (of the numpy array) is a type of chess piece of the board itself.
# The value of the index is either a 0 or a 1, indicating if the type of chess
# piece is there. For example, the map is as follows:
# ""class_mappings = {
#     'board': 0,
#     'b-bishop': 1,
#     'b-king': 2,
#     'b-knight': 3,
#     'b-pawn': 4,
#     'b-queen': 5,
#     'b-rook': 6
# }
# ""
# The following cell is parses an entire folder of .xml files (the annotations).
# The output is a numpy array of labels. This will be passed into the parameter
# for stratified k fold technique to ensure even distribution of representation
# of classes are within the training and validation batches.

import os
import xml.etree.ElementTree as ET
import numpy as np

# Path to the folder containing all the XML annotation files
annotations_folder = '/content/drive/MyDrive/images_white_pieces_strain/annotations'

# # Updated class mappings
class_mappings = {
    'board': 0,
    'w-bishop': 1,
    'w-king': 2,
    'w-knight': 3,
    'w-pawn': 4,
    'w-queen': 5,
    'w-rook': 6
}
# class_mappings = {
#     'b-bishop': 0,
#     'b-king': 1,
#     'b-knight': 2,
#     'b-pawn': 3,
#     'b-queen': 4,
#     'b-rook': 5,
#     'board': 6,
#     'w-bishop': 7,
#     'w-king': 8,
#     'w-knight': 9,
#     'w-pawn': 10,
#     'w-queen': 11,
#     'w-rook': 12
# }

# def extract_labels_from_xml(xml_file, class_mappings):
#     # Parse the XML file
#     tree = ET.parse(xml_file)
#     root = tree.getroot()

#     # Initialize an array to represent the labels for the image
#     image_labels = [0] * len(class_mappings)  # Initialize based on number of classes

#     # Iterate through each object in the XML
#     for obj in root.findall('.//object'):
#         label = obj.find('name').text
#         # Update the label array if the label is in class_mappings
#         if label in class_mappings:
#             image_labels[class_mappings[label]] = 1

#     return image_labels

# # Initialize an empty list to store the labels for all images
# all_labels = []

# # Iterate over all XML files in the annotations folder
# for filename in os.listdir(annotations_folder):
#     if filename.endswith('.xml'):  # Check if the file is an XML file
#         xml_file = os.path.join(annotations_folder, filename)

#         # Extract labels for the current XML file
#         labels_for_image = extract_labels_from_xml(xml_file, class_mappings)

#         # Append the labels to the list of all labels
#         all_labels.append(labels_for_image)

# # Convert the list of all labels into a numpy array
# labels = np.array(all_labels)  # This will be a 2D array

# # New Requirement: Create a numpy array with the sequence [0, 1, 2, 3, 4, 5, 6] for each row
# # The number of rows should match the number of rows in the `labels` array
# sequence_array = np.tile(np.arange(len(class_mappings)), (labels.shape[0], 1))


import os
import numpy as np
import xml.etree.ElementTree as ET

# Assuming your class_mappings is defined
# class_mappings = {
#     'b-bishop': 0,
#     'b-king': 1,
#     'b-knight': 2,
#     'b-pawn': 3,
#     'b-queen': 4,
#     'b-rook': 5,
#     'board': 6,
#     'w-bishop': 7,
#     'w-king': 8,
#     'w-knight': 9,
#     'w-pawn': 10,
#     'w-queen': 11,
#     'w-rook': 12
# }

def extract_labels_from_xml(xml_file, class_mappings):
    # Parse the XML file
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Initialize an array to represent the labels for the image
    image_labels = [0] * len(class_mappings)  # Initialize based on the number of classes

    # Iterate through each object in the XML
    for obj in root.findall('.//object'):
        label = obj.find('name').text
        # Update the label array if the label is in class_mappings
        if label in class_mappings:
            image_labels[class_mappings[label]] = 1

    return image_labels

# Initialize an empty list to store the labels for all images
all_labels = []

# Iterate over all XML files in the annotations folder
for filename in os.listdir(annotations_folder):
    if filename.endswith('.xml'):  # Check if the file is an XML file
        xml_file = os.path.join(annotations_folder, filename)

        # Extract labels for the current XML file
        labels_for_image = extract_labels_from_xml(xml_file, class_mappings)

        # Append the labels to the list of all labels
        all_labels.append(labels_for_image)

# Convert the list of all labels into a numpy array
labels = np.array(all_labels)  # This will be a 2D array

# Create a numpy array with the sequence [0, 1, 2, 3, 4, 5, 6] for each row
# The number of rows should match the number of rows in the `labels` array
sequence_array = np.tile(np.arange(len(class_mappings)), (labels.shape[0], 1))

print(labels.shape)
print(sequence_array.shape)


print(labels)

print(sequence_array)


(1301, 7)
(1301, 7)
[[1 1 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 [1 0 1 ... 0 1 0]
 ...
 [1 0 1 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 [1 1 0 ... 0 0 0]]
[[0 1 2 ... 4 5 6]
 [0 1 2 ... 4 5 6]
 [0 1 2 ... 4 5 6]
 ...
 [0 1 2 ... 4 5 6]
 [0 1 2 ... 4 5 6]
 [0 1 2 ... 4 5 6]]


The following code below outputs if there are any objects with naming inconsistencies for objects within an .xml file.

In [ ]:
import os
import xml.etree.ElementTree as ET

def check_xml_files(folder_path):
    xml_files = [f for f in os.listdir(folder_path) if f.endswith('.xml')]

    for xml_file in xml_files:
        xml_path = os.path.join(folder_path, xml_file)

        # Parse the XML file
        tree = ET.parse(xml_path)
        root = tree.getroot()

        # Check for objects named "w-knight", "w-pawn", or "rotation"
        object_names = [obj.find('name').text for obj in root.findall('.//object')]
        if 'white-knight' in object_names:
          # if 'w-knight' in object_names or 'w-pawn' in object_names or 'rotation' in object_names:
            print(f"File {xml_file} contains 'white-knight', 'white-pawn', or 'rotation' objects.")

# Example usage
folder_path = '/content/drive/MyDrive/Stratified/Stratified_White_Dataset/Validation/annotations'
check_xml_files(folder_path)


The following below code checks if "board" objects is not contained in any of the .xml files.

In [ ]:
import os
import xml.etree.ElementTree as ET

def check_xml_files(folder_path):
    xml_files = [f for f in os.listdir(folder_path) if f.endswith('.xml')]

    for xml_file in xml_files:
        xml_path = os.path.join(folder_path, xml_file)

        # Parse the XML file
        tree = ET.parse(xml_path)
        root = tree.getroot()

        # Check if "board" object is not present
        object_names = [obj.find('name').text for obj in root.findall('.//object')]
        if 'board' not in object_names:
            print(f"File {xml_file} does not contain 'board' object.")

# Example usage
folder_path = '/content/non-stratified_chess-set-2/valid/labels'
check_xml_files(folder_path)


In [ ]:
print(labels)

[[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [0 1 0 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]]


The below code is basically a modified, forked GitHub repo. Here is the GitHub repo for the implementation of an algorithm which optimizes data splitting to further improve our object detection model:

https://github.com/trent-b/iterative-stratification

This algorithm is known as Multilabel Stratified K-Fold.

This was an intentional design choice to include this to ensure that there is an even distribution of representation among classes for processing data that could contain multi-class and multi-label images.

Run the below cell code block when you are preparing to data split.



In [ ]:
import numpy as np

from sklearn.utils import check_random_state
from sklearn.utils.validation import _num_samples, check_array
from sklearn.utils.multiclass import type_of_target

from sklearn.model_selection._split import _BaseKFold, _RepeatedSplits, \
BaseShuffleSplit, _validate_shuffle_split

# IterativeStratification() is key for achieving stratification for multilabel
# data.

def IterativeStratification(labels, r, random_state):
    """This function implements the Iterative Stratification algorithm described
    in the following paper:
    Sechidis K., Tsoumakas G., Vlahavas I. (2011) On the Stratification of
    Multi-Label Data. In: Gunopulos D., Hofmann T., Malerba D., Vazirgiannis M.
    (eds) Machine Learning and Knowledge Discovery in Databases. ECML PKDD
    2011. Lecture Notes in Computer Science, vol 6913. Springer, Berlin,
    Heidelberg.
    """

    n_samples = labels.shape[0]
    test_folds = np.zeros(n_samples, dtype=int)

    # Calculate the desired number of examples at each subset
    c_folds = r * n_samples

    # Calculate the desired number of examples of each label at each subset
    c_folds_labels = np.outer(r, labels.sum(axis=0))

    labels_not_processed_mask = np.ones(n_samples, dtype=bool)

    while np.any(labels_not_processed_mask):
        # Find the label with the fewest (but at least one) remaining examples,
        # breaking ties randomly
        num_labels = labels[labels_not_processed_mask].sum(axis=0)

        # Handle case where only all-zero labels are left by distributing
        # across all folds as evenly as possible (not in original algorithm but
        # mentioned in the text). (By handling this case separately, some
        # code redundancy is introduced; however, this approach allows for
        # decreased execution time when there are a relatively large number
        # of all-zero labels.)
        if num_labels.sum() == 0:
            sample_idxs = np.where(labels_not_processed_mask)[0]

            for sample_idx in sample_idxs:
                fold_idx = np.where(c_folds == c_folds.max())[0]

                if fold_idx.shape[0] > 1:
                    fold_idx = fold_idx[random_state.choice(fold_idx.shape[0])]

                test_folds[sample_idx] = fold_idx
                c_folds[fold_idx] -= 1

            break

        label_idx = np.where(num_labels == num_labels[np.nonzero(num_labels)].min())[0]
        if label_idx.shape[0] > 1:
            label_idx = label_idx[random_state.choice(label_idx.shape[0])]

        sample_idxs = np.where(np.logical_and(labels[:, label_idx].flatten(), labels_not_processed_mask))[0]
        for sample_idx in sample_idxs:
    # Find the subset(s) with the largest number of desired examples
    # for this label, breaking ties by considering the largest number
    # of desired examples, breaking further ties randomly

          label_folds = c_folds_labels[:, label_idx]
          fold_idx = np.where(label_folds == label_folds.max())[0]

          if fold_idx.shape[0] > 1:
              temp_fold_idx = np.where(c_folds[fold_idx] == c_folds[fold_idx].max())[0]
              if temp_fold_idx.shape[0] > 1:
                  # Choose one randomly if there are multiple max values
                  fold_idx = random_state.choice(fold_idx[temp_fold_idx])
              else:
                  # If temp_fold_idx has only one element, directly use this value
                  fold_idx = fold_idx[temp_fold_idx[0]]
          else:
              # If fold_idx has only one element, directly use this value
              fold_idx = fold_idx[0]

          test_folds[sample_idx] = fold_idx
          labels_not_processed_mask[sample_idx] = False

          # Update desired number of examples
          c_folds_labels[fold_idx, :] -= labels[sample_idx, :]
          c_folds[fold_idx] -= 1


        return test_folds

The below code is a specific type of implementation of stratification algorithm. This code calls its base class function. And gives explanations of how to implement this for a unique project.

Run the below cell code block when you are preparing to data split.

In [ ]:
class MultilabelStratifiedKFold(_BaseKFold):
    """Multilabel stratified K-Folds cross-validator
    Provides train/test indices to split multilabel data into train/test sets.
    This cross-validation object is a variation of KFold that returns
    stratified folds for multilabel data. The folds are made by preserving
    the percentage of samples for each label.
    Parameters
    ----------
    n_splits : int, default=3
        Number of folds. Must be at least 2.
    shuffle : boolean, optional
        Whether to shuffle each stratification of the data before splitting
        into batches.
    random_state : int, RandomState instance or None, optional, default=None
        If int, random_state is the seed used by the random number generator;
        If RandomState instance, random_state is the random number generator;
        If None, the random number generator is the RandomState instance used
        by `np.random`. Unlike StratifiedKFold that only uses random_state
        when ``shuffle`` == True, this multilabel implementation
        always uses the random_state since the iterative stratification
        algorithm breaks ties randomly.
    Examples
    --------
    >>> from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
    >>> import numpy as np
    >>> X = np.array([[1,2], [3,4], [1,2], [3,4], [1,2], [3,4], [1,2], [3,4]])
    >>> y = np.array([[0,0], [0,0], [0,1], [0,1], [1,1], [1,1], [1,0], [1,0]])
    >>> mskf = MultilabelStratifiedKFold(n_splits=2, random_state=0)
    >>> mskf.get_n_splits(X, y)
    2
    >>> print(mskf)  # doctest: +NORMALIZE_WHITESPACE
    MultilabelStratifiedKFold(n_splits=2, random_state=0, shuffle=False)
    >>> for train_index, test_index in mskf.split(X, y):
    ...    print("TRAIN:", train_index, "TEST:", test_index)
    ...    X_train, X_test = X[train_index], X[test_index]
    ...    y_train, y_test = y[train_index], y[test_index]
    TRAIN: [0 3 4 6] TEST: [1 2 5 7]
    TRAIN: [1 2 5 7] TEST: [0 3 4 6]
    Notes
    -----
    Train and test sizes may be slightly different in each fold.
    See also
    --------
    RepeatedMultilabelStratifiedKFold: Repeats Multilabel Stratified K-Fold
    n times.
    """

    def __init__(self, n_splits=3, *, shuffle=False, random_state=None):
        super(MultilabelStratifiedKFold, self).__init__(n_splits=n_splits, shuffle=shuffle, random_state=random_state)

    def _make_test_folds(self, X, y):
      y = np.asarray(y, dtype=bool)
      type_of_target_y = type_of_target(y)

      if type_of_target_y != 'multilabel-indicator':
          raise ValueError('Supported target type is: multilabel-indicator. Got {!r} instead.'.format(type_of_target_y))

      num_samples = y.shape[0]
      rng = check_random_state(self.random_state)
      indices = np.arange(num_samples)

      if self.shuffle:
          rng.shuffle(indices)
          y = y[indices]

      r = np.asarray([1 / self.n_splits] * self.n_splits)

      test_folds = IterativeStratification(labels=y, r=r, random_state=rng)

      # Ensure fold_idx is a scalar
      # This line or similar logic needs modification
      # For example, when selecting fold_idx, ensure it's a single value:
      # fold_idx = fold_idx[0] if isinstance(fold_idx, np.ndarray) and fold_idx.size > 0 else fold_idx

      return test_folds[np.argsort(indices)]


    def _iter_test_masks(self, X=None, y=None, groups=None):
        test_folds = self._make_test_folds(X, y)
        for i in range(self.n_splits):
            yield test_folds == i

    def split(self, X, y, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
            Note that providing ``y`` is sufficient to generate the splits and
            hence ``np.zeros(n_samples)`` may be used as a placeholder for
            ``X`` instead of actual training data.
        y : array-like, shape (n_samples, n_labels)
            The target variable for supervised learning problems.
            Multilabel stratification is done based on the y labels.
        groups : object
            Always ignored, exists for compatibility.
        Returns
        -------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        Notes
        -----
        Randomized CV splitters may return different results for each call of
        split. You can make the results identical by setting ``random_state``
        to an integer.
        """
        y = check_array(y, ensure_2d=False, dtype=None)
        return super(MultilabelStratifiedKFold, self).split(X, y, groups)


Run the below code after the two previous code blocks have been run. You can specify the number of splits, whether or  not you are in a random state, or if you want to specify shuffling.

In [ ]:
mskf = MultilabelStratifiedKFold(n_splits=2, random_state=0, shuffle=True)


In [ ]:
print("sequence_array shape:", sequence_array.shape)
print("labels shape:", labels.shape)



sequence_array shape: (1301, 7)
labels shape: (1301, 7)


The below code's output: returns a list of indices of the original  dataset to now put into training and validation batches.

What we are now capable with running this code: we know which images to put into the training and validation batches--based on the indices of the original dataset.


In [ ]:
for train_index, val_index in mskf.split(sequence_array, labels):
    # Converting numpy arrays to lists for string joining
    train_index_list = list(train_index)
    val_index_list = list(val_index)

    # Joining the list elements with a comma and space for readability
    train_indices_str = ", ".join(map(str, train_index_list))
    test_indices_str = ", ".join(map(str, val_index_list))

    print("TRAIN:", train_indices_str, "VALIDATION:", test_indices_str)
    X_train, X_test = sequence_array[train_index], sequence_array[val_index]
    y_train, y_test = labels[train_index], labels[val_index]



TRAIN: 658, 665, 677, 683, 684, 688, 691, 692, 695, 697, 701, 705, 708, 710, 715, 725, 727, 731, 735, 737, 743, 744, 748, 751, 752, 759, 760, 761, 762, 763, 768, 771, 773, 775, 777, 779, 780, 781, 782, 783, 784, 788, 791, 792, 795, 800, 802, 807, 808, 811, 815, 816, 823, 824, 825, 826, 830, 833, 834, 836, 839, 843, 844, 851, 852, 858, 859, 862, 865, 872, 875, 876, 881, 886, 887, 888, 892, 893, 894, 895, 897, 898, 900, 902, 910, 911, 912, 914, 924, 926, 927, 929, 931, 932, 933, 935, 938, 941, 944, 945, 946, 947, 948, 956, 959, 964, 965, 966, 967, 971, 972, 976, 979, 982, 988, 990, 991, 993, 994, 996 VALIDATION: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 

The below code's output: if train and val folders do not exist in our base directory, base_dir, we specify, we create these folders to now copy over our images into the training and val batches based off the previous indices we get from previous code block's execution.

In [ ]:
# attempting to save training and testing (val) batches post-stratification
import os

# Define your base directory where you want to save the images
base_dir = '/content/drive/MyDrive/Stratified/Stratified_White_Gray_Dataset'

# Create training and testing directories
train_dir = os.path.join(base_dir, 'Train')
val_dir = os.path.join(base_dir, 'Validation')

# Making directories if they don't exist
if not os.path.exists(train_dir):
    os.makedirs(train_dir)
if not os.path.exists(val_dir):
    os.makedirs(val_dir)


The below code's output: based off of the indices specified we got from performing stratification, we can iterate through

The below code's output: making sure there are no joint images, repeat images, in folders to ensure that stratification algorithm implementation worked mechanically.

MAKING SURE THE ANNOTATIONS CORRESPOND TO THE IMAGES WHEN I UPLOAD TO ROBOFLOW
DO NOT RUN THIS MORE THAN ONCE. ONLY RUN IF YOU  HAVE ZERO IMAGES IN

The below code's output: images are put into target folder, val folder or train folder. This is actually how the training and validation images appear in the folders.

Because we iterate through the annotation folder, checking the name of each file within the annotation folder, and we put the corresponding .jpg image in with the annotation name to ENSURE that annotations are matching up with their images in each of the val and training folders.

In [ ]:
# # List to hold the paths of all the image ANNOTATIONS
# annotation_paths = []
# source_annotation_dir = '/content/drive/MyDrive/images_gray_pieces_strain/annotations/Annotations'

# # Walk through the source directory and append each image path to the list
# for root, dirs, files in os.walk(source_annotation_dir):
#     for file in files:
#         if file.endswith('.xml'):  # Add or modify the condition based on your image file types
#             annotation_paths.append(os.path.join(root, file))

# print(annotation_paths)

import os

# List to hold the paths of all the image ANNOTATIONS
annotation_paths = []
source_annotation_dir = '/content/drive/MyDrive/images_white_pieces_strain/annotations'

# Walk through the source directory and append each image path to the list
for root, dirs, files in os.walk(source_annotation_dir):
    for file in files:
        if file.endswith('.xml'):  # Add or modify the condition based on your image file types
            annotation_paths.append(os.path.join(root, file))

# Sort the annotation paths list
annotation_paths.sort()

print(annotation_paths)


['/content/drive/MyDrive/images_white_pieces_strain/annotations/WIN_20240207_12_07_19_Pro.xml', '/content/drive/MyDrive/images_white_pieces_strain/annotations/WIN_20240207_12_07_27_Pro.xml', '/content/drive/MyDrive/images_white_pieces_strain/annotations/WIN_20240207_12_07_31_Pro.xml', '/content/drive/MyDrive/images_white_pieces_strain/annotations/WIN_20240207_12_07_35_Pro.xml', '/content/drive/MyDrive/images_white_pieces_strain/annotations/WIN_20240207_12_07_42_Pro.xml', '/content/drive/MyDrive/images_white_pieces_strain/annotations/WIN_20240207_12_07_47_Pro.xml', '/content/drive/MyDrive/images_white_pieces_strain/annotations/WIN_20240207_12_07_51_Pro.xml', '/content/drive/MyDrive/images_white_pieces_strain/annotations/WIN_20240207_12_07_55_Pro.xml', '/content/drive/MyDrive/images_white_pieces_strain/annotations/WIN_20240207_12_07_59_Pro.xml', '/content/drive/MyDrive/images_white_pieces_strain/annotations/WIN_20240207_12_08_03_Pro.xml', '/content/drive/MyDrive/images_white_pieces_strai

In [ ]:
import os
import shutil

# Assuming TRAIN and VALIDATION are your arrays of indices
TRAIN = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 659, 660, 661, 662, 663, 664, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 678, 679, 680, 681, 682, 685, 686, 687, 689, 690, 693, 694, 696, 698, 699, 700, 702, 703, 704, 706, 707, 709, 711, 712, 713, 714, 716, 717, 718, 719, 720, 721, 722, 723, 724, 726, 728, 729, 730, 732, 733, 734, 736, 738, 739, 740, 741, 742, 745, 746, 747, 749, 750, 753, 754, 755, 756, 757, 758, 764, 765, 766, 767, 769, 770, 772, 774, 776, 778, 785, 786, 787, 789, 790, 793, 794, 796, 797, 798, 799, 801, 803, 804, 805, 806, 809, 810, 812, 813, 814, 817, 818, 819, 820, 821, 822, 827, 828, 829, 831, 832, 835, 837, 838, 840, 841, 842, 845, 846, 847, 848, 849, 850, 853, 854, 855, 856, 857, 860, 861, 863, 864, 866, 867, 868, 869, 870, 871, 873, 874, 877, 878, 879, 880, 882, 883, 884, 885, 889, 890, 891, 896, 899, 901, 903, 904, 905, 906, 907, 908, 909, 913, 915, 916, 917, 918, 919, 920, 921, 922, 923, 925, 928, 930, 934, 936, 937, 939, 940, 942, 943, 949, 950, 951, 952, 953, 954, 955, 957, 958, 960, 961, 962, 963, 968, 969, 970, 973, 974, 975, 977, 978, 980, 981, 983, 984, 985, 986, 987, 989, 992, 995, 997, 998, 999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034, 1035, 1036, 1037, 1038, 1039, 1040, 1041, 1042, 1043, 1044, 1045, 1046, 1047, 1048, 1049, 1050, 1051, 1052, 1053, 1054, 1055, 1056, 1057, 1058, 1059, 1060, 1061, 1062, 1063, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1078, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1096, 1097, 1098, 1099, 1100, 1101, 1102, 1103, 1104, 1105, 1106, 1107, 1108, 1109, 1110, 1111, 1112, 1113, 1114, 1115, 1116, 1117, 1118, 1119, 1120, 1121, 1122, 1123, 1124, 1125, 1126, 1127, 1128, 1129, 1130, 1131, 1132, 1133, 1134, 1135, 1136, 1137, 1138, 1139, 1140, 1141, 1142, 1143, 1144, 1145, 1146, 1147, 1148, 1149, 1150, 1151, 1152, 1153, 1154, 1155, 1156, 1157, 1158, 1159, 1160, 1161, 1162, 1163, 1164, 1165, 1166, 1167, 1168, 1169, 1170, 1171, 1172, 1173, 1174, 1175, 1176, 1177, 1178, 1179, 1180, 1181, 1182, 1183, 1184, 1185, 1186, 1187, 1188, 1189, 1190, 1191, 1192, 1193, 1194, 1195, 1196, 1197, 1198, 1199, 1200, 1201, 1202, 1203, 1204, 1205, 1206, 1207, 1208, 1209, 1210, 1211, 1212, 1213, 1214, 1215, 1216, 1217, 1218, 1219, 1220, 1221, 1222, 1223, 1224, 1225, 1226, 1227, 1228, 1229, 1230, 1231, 1232, 1233, 1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243, 1244, 1245, 1246, 1247, 1248, 1249, 1250, 1251, 1252, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 1260, 1261, 1262, 1263, 1264, 1265, 1266, 1267, 1268, 1269, 1270, 1271, 1272, 1273, 1274, 1275, 1276, 1277, 1278, 1279, 1280, 1281, 1282, 1283, 1284, 1285, 1286, 1287, 1288, 1289, 1290, 1291, 1292, 1293, 1294, 1295, 1296, 1297, 1298, 1299, 1300 ]

VALIDATE = [658, 665, 677, 683, 684, 688, 691, 692, 695, 697, 701, 705, 708, 710, 715, 725, 727, 731, 735, 737, 743, 744, 748, 751, 752, 759, 760, 761, 762, 763, 768, 771, 773, 775, 777, 779, 780, 781, 782, 783, 784, 788, 791, 792, 795, 800, 802, 807, 808, 811, 815, 816, 823, 824, 825, 826, 830, 833, 834, 836, 839, 843, 844, 851, 852, 858, 859, 862, 865, 872, 875, 876, 881, 886, 887, 888, 892, 893, 894, 895, 897, 898, 900, 902, 910, 911, 912, 914, 924, 926, 927, 929, 931, 932, 933, 935, 938, 941, 944, 945, 946, 947, 948, 956, 959, 964, 965, 966, 967, 971, 972, 976, 979, 982, 988, 990, 991, 993, 994, 996]

train_annotation_dir = '/content/drive/MyDrive/Stratified/Stratified_White_Dataset/Train/annotations'
val_annotation_dir = '/content/drive/MyDrive/Stratified/Stratified_White_Dataset/Validation/annotations'

def copy_ants(indices, source_paths, dest_dir):
    for idx in indices:
        # Define source and destination paths
        src_path = source_paths[idx]
        dest_path = os.path.join(dest_dir, os.path.basename(src_path))

        # Copy the image
        shutil.copy(src_path, dest_path)

# Copy annotations from original annotations to train folder
copy_ants(TRAIN, annotation_paths, train_annotation_dir)

# Copy annotations from original annotations to validate folder
copy_ants(VALIDATE, annotation_paths, val_annotation_dir)


In [ ]:
import os
import shutil

# Paths to the folders
images_folder = '/content/drive/MyDrive/images_white_pieces_strain/raw'
annotations_folder = '/content/drive/MyDrive/Stratified/Stratified_White_Dataset/Validation/annotations'
target_folder = '/content/drive/MyDrive/Stratified/Stratified_White_Dataset/Validation/raw'

# Get a sorted list of annotation files
annotation_files = sorted([f for f in os.listdir(annotations_folder) if f.endswith('.xml')])

# Iterate through sorted annotation files
for annotation_file in annotation_files:
    # Extract base filename without extension
    base_filename = os.path.splitext(annotation_file)[0]

    # Check if corresponding image file exists
    image_file = os.path.join(images_folder, base_filename + '.jpg')
    if os.path.exists(image_file):
        # Copy image file to target folder
        shutil.copy(image_file, os.path.join(target_folder, base_filename + '.jpg'))



The below function takes a file path to a folder and checks if there are any duplicates in this folder.

In [ ]:
import os

def find_duplicates_by_basename(folder_path):
    file_dict = {}

    for root, dirs, files in os.walk(folder_path):
        for file_name in files:
            file_path = os.path.join(root, file_name)
            base_name = os.path.splitext(file_name)[0]

            # Check for duplicates based on base name
            if base_name in file_dict:
                file_dict[base_name].append(file_path)
            else:
                file_dict[base_name] = [file_path]

    # Filter out non-duplicates
    duplicate_files = {k: v for k, v in file_dict.items() if len(v) > 1}

    return duplicate_files

# Example usage
folder_path = '/content/drive/MyDrive/images_white_pieces_strain/annotations/Annotations'
duplicates = find_duplicates_by_basename(folder_path)

if duplicates:
    print("Duplicate files found:")
    for base_name, file_paths in duplicates.items():
        print(f"Base Name: {base_name}")
        for file_path in file_paths:
            print(f"- {file_path}")
else:
    print("No duplicate files found.")



No duplicate files found.


In [ ]:
# CHECKING TO MAKE SURE THE STRATIFIED SPLIT DIDN'T SCREW UP OMGGGGG

import os

def find_duplicates(folder1, folder2):
    # Get list of filenames in each folder
    files1 = set(os.listdir(folder1))
    files2 = set(os.listdir(folder2))

    # Find intersection of filenames (i.e., duplicates)
    duplicates = files1.intersection(files2)

    return duplicates

# Paths to the folders
folder1_path = '/content/drive/MyDrive/Stratified_Gray_Dataset/Validation/raw'
folder2_path = '/content/drive/MyDrive/Stratified_Gray_Dataset/Train/raw'

# Find duplicates
duplicates = find_duplicates(folder1_path, folder2_path)

# Print the duplicate filenames
if duplicates:
    print("Duplicate files found:")
    for filename in duplicates:
        print(filename)
else:
    print("No duplicate files found.")


No duplicate files found.


The below code's output: a folder with all the annotation paths from another folder containing annotations. Pretty much, we can a list of annotation file paths from a folder containing annotations.

NEEEEEEEEEEEEEEDED*

Stratification

The below code:

TAKING THE INDICES POST-STRATIFICATION AND PUTTING THE ORIGINAL ANNOTATIONS
INTO THE APPROPRIATE TESTING AND VALIDATION BATCHES

Function to copy images to a new directory

Stratification:

The below code's output: produces the count for how many files are contained in an arbitrary folder.

In [ ]:
# CHECKS WHETHER OR NOT DATA GOT SPLIT BC IM COUNTING HOW MANY IMAGES ARE IN THE
# TEST/TRAIN FOLDER(S).
import os

# Define your source directory containing the images
source_dir = '/content/drive/MyDrive/images_white_pieces_strain/annotations'

# Initialize a counter
image_count = 0

# Walk through the source directory to count image files
for root, dirs, files in os.walk(source_dir):
    for file in files:
        if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.xml', '.gif')):  # Include other image formats as needed
            image_count += 1

print(f"Total number of images in the folder: {image_count}")


Total number of images in the folder: 1301


In [ ]:
import os

def find_mismatched_files(image_folder, annotation_folder):
    image_files = [f for f in os.listdir(image_folder) if f.endswith('.jpg')]
    annotation_files = [f for f in os.listdir(annotation_folder) if f.endswith('.xml')]

    mismatched_image_files = set()
    mismatched_xml_files = set()

    # Find mismatched files
    for image_file in image_files:
        xml_file = image_file.replace('.jpg', '.xml')
        if xml_file not in annotation_files or not os.path.isfile(os.path.join(annotation_folder, xml_file)):
            mismatched_image_files.add(image_file)

    for xml_file in annotation_files:
        image_file = xml_file.replace('.xml', '.jpg')
        if image_file not in image_files or not os.path.isfile(os.path.join(image_folder, image_file)):
            mismatched_xml_files.add(xml_file)

    return mismatched_image_files, mismatched_xml_files

def delete_files(folder, files):
    for file in files:
        file_path = os.path.join(folder, file)
        os.remove(file_path)
        print(f"Deleted: {file_path}")

# Example usage
image_folder_path = '/content/drive/MyDrive/Stratified/Stratified_White_Dataset/Validation/raw'
annotation_folder_path = '/content/drive/MyDrive/Stratified/Stratified_White_Dataset/Validation/annotations'

mismatched_images, mismatched_xmls = find_mismatched_files(image_folder_path, annotation_folder_path)

print("Mismatched Image Files:")
print(mismatched_images)

print("\nMismatched XML Files:")
print(mismatched_xmls)

# Delete mismatched files
# delete_files(image_folder_path, mismatched_images)
# delete_files(annotation_folder_path, mismatched_xmls)


Mismatched Image Files:
set()

Mismatched XML Files:
set()


The below code checks if the indices of files in two folders correspond to the same base file name--we iterate through both folders simultaneously and compare the base names of files at each index.

In [ ]:
import os

def check_corresponding_base_names(folder1, folder2):
    files1 = sorted([f for f in os.listdir(folder1) if os.path.isfile(os.path.join(folder1, f))])
    files2 = sorted([f for f in os.listdir(folder2) if os.path.isfile(os.path.join(folder2, f))])

    if len(files1) != len(files2):
        print("The number of files in the two folders is different.")
        return

    for file1, file2 in zip(files1, files2):
        base_name1, ext1 = os.path.splitext(file1)
        base_name2, ext2 = os.path.splitext(file2)

        if base_name1 != base_name2:
            print(f"Base names do not match at index {files1.index(file1)}:")
            print(f"{file1} in folder1, {file2} in folder2")
        # else:
            # print(f"Base names match at index {files1.index(file1)}:")
            # print(f"{file1} in folder1, {file2} in folder2")

# Example usage
folder_path1 = '/content/drive/MyDrive/images_white_pieces_strain/raw'
folder_path2 = '/content/drive/MyDrive/images_white_pieces_strain/annotations/Annotations'

check_corresponding_base_names(folder_path1, folder_path2)


# Augmentation

The below cells attempts to perform image augmentation.

In [ ]:
# PRE-PROCESSING + AUGMENTATION OF IMAGES
# 1. Import necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# LEFT OFF ON https://www.youtube.com/watch?v=WSvpLUietIM

In [ ]:
# Image dimensions from the XML
image_width = 1920
image_height = 1080

# Normalizing b-pawn bounding box
xmin_normalized_pawn = 1377.2 / image_width
ymin_normalized_pawn = 86.7 / image_height
xmax_normalized_pawn = 1461.31 / image_width
ymax_normalized_pawn = 165.9 / image_height

# Normalizing board bounding box
xmin_normalized_board = 489.2 / image_width
ymin_normalized_board = 29.28 / image_height
xmax_normalized_board = 1494.71 / image_width
ymax_normalized_board = 1035.68 / image_height

# Normalized bounding boxes
bboxes_normalized = [
    [ymin_normalized_pawn, xmin_normalized_pawn, ymax_normalized_pawn, xmax_normalized_pawn],
    [ymin_normalized_board, xmin_normalized_board, ymax_normalized_board, xmax_normalized_board]
]

print("Normalized Bounding Boxes:")
print("b-pawn:", bboxes_normalized[0])
print("board:", bboxes_normalized[1])


Normalized Bounding Boxes:
b-pawn: [0.08027777777777778, 0.7172916666666667, 0.15361111111111111, 0.7610989583333333]
board: [0.027111111111111114, 0.25479166666666664, 0.958962962962963, 0.7784947916666667]


In [ ]:
# 2. Setup Directories
bbox = bboxes_normalized
bbox_tensor = tf.convert_to_tensor(bbox, dtype=tf.float32)


# Example of loading an image with TensorFlow
image_path = '/content/drive/MyDrive/images_gray_pieces_strain/raw/WIN_20240221_19_01_10_Pro.jpg'
image = tf.io.read_file(image_path)
image = tf.image.decode_jpeg(image, channels=3)  # Ensure 'channels=3' for RGB images


In [ ]:
# Practice flipping horizontally
def flip_image_horizontal(image, bbox):
    flipped_image = tf.image.flip_left_right(image)
    # Adjust bounding box for horizontal flip
    # [y_min, x_min, y_max, x_max] -> [y_min, 1 - x_max, y_max, 1 - x_min]
    flipped_bbox = tf.stack([bbox[:, 0], 1 - bbox[:, 3], bbox[:, 2], 1 - bbox[:, 1]], axis=-1)
    return flipped_image, flipped_bbox

In [ ]:
flipped_image, flipped_bboxes = flip_image_horizontal(image, bbox_tensor)

print(flipped_image)

tf.Tensor(
[[[ 0  2  1]
  [ 0  2  1]
  [ 0  2  1]
  ...
  [76 90 90]
  [76 92 89]
  [77 93 90]]

 [[ 0  2  1]
  [ 0  2  1]
  [ 0  2  1]
  ...
  [76 90 90]
  [75 91 88]
  [75 91 88]]

 [[ 0  2  1]
  [ 0  2  1]
  [ 0  2  1]
  ...
  [75 91 88]
  [74 90 87]
  [74 90 87]]

 ...

 [[53 68 63]
  [53 68 63]
  [53 70 64]
  ...
  [59 78 76]
  [57 77 75]
  [57 77 75]]

 [[53 70 64]
  [53 70 64]
  [52 71 65]
  ...
  [57 76 74]
  [55 75 73]
  [55 75 73]]

 [[54 71 65]
  [54 71 65]
  [52 71 65]
  ...
  [56 75 73]
  [54 74 72]
  [54 74 72]]], shape=(1080, 1920, 3), dtype=uint8)


# Importing dataset which has already been split.

The below code assumes that we have already performed stratification for data splitting.

In [ ]:
# Purpose:
# This command installs the Roboflow package, a tool that helps in managing,
# preprocessing, and augmenting datasets for machine learning, especially for
# computer vision tasks.
#!pip install roboflow

# Imports the Roboflow library into the Python environment.
# from roboflow import Roboflow
# # Initializes the Roboflow object with an API key, allowing authenticated access
# # to your Roboflow account and its functionalities.
# rf = Roboflow(api_key="PpFunxi8rDrNKv1MlsLN")
# # Selects a specific project within a workspace. In this case, the project is
# # presumably related to chess piece images, which would be used for training the
# # model.
# THE FOLLOWING CODE IMPORTS THE PROCESSED DATA TO BE ABLE TO RUN INFERENCE

# Import already-curated dataset of chess images

!mkdir {HOME}/datasets
%cd {HOME}/datasets
# !pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="Q8STLvZqQ864nfFT4e7U")
# project = rf.workspace("senior-project-vwmvt").project("chess-detection-arsbp")
# version = project.version(3)
# dataset = version.download("yolov8")

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="Q8STLvZqQ864nfFT4e7U")
project = rf.workspace("senior-project-vwmvt").project("random-chess-detection")
version = project.version(1)
dataset = version.download("yolov8")




mkdir: cannot create directory ‘{HOME}/datasets’: No such file or directory
[Errno 2] No such file or directory: '{HOME}/datasets'
/content
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.1.23, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to random-chess-detection-1 in yolov8:: 100%|██████████| 16497/16497 [00:05<00:00, 2850.84it/s]


# Checking frequency of classes among batches

The below code's output: logic to see the frequency of how many times an class appears in an image. so
if a there are two queens that show up in an image, it will only mark this
queen showing up once for this image.

In [ ]:
import xml.etree.ElementTree as ET
import glob

# Function to parse a single XML file and return classes present
def parse_xml_file(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    classes_present = set()  # Use a set to store unique class names present in the file
    for obj in root.findall('object'):
        class_name = obj.find('name').text
        classes_present.add(class_name)
    return classes_present

# Initialize a dictionary to store the count of images each class appears in
class_image_count = {}

# List all XML files in the folder
xml_files = glob.glob('/content/drive/MyDrive/images_gray_pieces_strain/annotations/Annotations/*.xml')

# Iterate over each XML file
for file_path in xml_files:
    classes_present = parse_xml_file(file_path)
    for class_name in classes_present:
        if class_name not in class_image_count:
            class_image_count[class_name] = 0
        class_image_count[class_name] += 1

# Print the number of images each class appears in
for class_name, count in class_image_count.items():
    print(f"{class_name}: freq. {count}")


board: freq. 1100
b-king: freq. 301
b-pawn: freq. 299
b-knight: freq. 419
b-queen: freq. 260
b-rook: freq. 339
b-bishop: freq. 388


The below code counts the TOTAL frequencies of each type of chess piece within an image. So, if there are 12 pawns in an image, the frequency would be 12 pawns for this image. This is different than our last code block--because our last code block counts the frequency if a class is present within an image. So the last code block's logic would count 1 frequency of a pawn within an image, even if there were 12 pawns present.

HERE IS THE FUNCTION TO PARSE THE FOLDER OF .XML FILES TO DETERMINE THE FREQUENCIES OF TYPES OF CHESS PIECES ACROSS ALL IMAGES.

In [ ]:
# HERE IS THE FUNCTION TO PARSE THE FOLDER OF .XML FILES TO DETERMINE THE
# FREQUENCIES OF TYPES OF CHESS PIECES ACROSS ALL IMAGES.

import xml.etree.ElementTree as ET
import glob

# Function to parse a single XML file and update total class frequencies
def update_class_frequencies_from_file(file_path, class_frequencies):
    # Parse the XML file
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Update class frequencies based on this file
    for obj in root.findall('object'):
        class_name = obj.find('name').text
        if class_name not in class_frequencies:
            class_frequencies[class_name] = 0
        class_frequencies[class_name] += 1

# Initialize a dictionary to store the total occurrences of each class across all images
total_class_frequencies = {}

# List all XML files in the folder
xml_files = glob.glob('/content/drive/MyDrive/images_white_pieces_strain/second_half_strain_images/annotations_second_half_whites/Annotations/*.xml')

# Iterate over each XML file and update class frequencies
for file_path in xml_files:
    update_class_frequencies_from_file(file_path, total_class_frequencies)

# Print the total occurrences of each class across all images
for class_name, frequency in total_class_frequencies.items():
    print(f"{class_name}: freq. {frequency}")


board: freq. 352
white-pawn: freq. 1242
white-rook: freq. 369
white-bishop: freq. 317
white-knight: freq. 370
white-king: freq. 178
white-queen: freq. 184


The below code's output: reuses the previous function for counting frequency. Specify which folder, batch, you want to use

In [ ]:
# Initialize a dictionary to store the total occurrences of each class across all images
# within val and train batches
val_total_class_frequencies = {}
train_total_class_frequencies = {}

# List all XML files in the folder
# looking at batch distribution to see if stratification worked.
# TEST batch
print("VAL batch\n")
xml_files = glob.glob('/content/drive/MyDrive/Dataset/test/annotations/*.xml')
# Iterate over each XML file and update class frequencies
for file_path in xml_files:
    update_class_frequencies_from_file(file_path, val_total_class_frequencies)

# Print the total occurrences of each class across all images
for class_name, frequency in val_total_class_frequencies.items():
    print(f"{class_name}: freq. {frequency}")

print("\nTRAIN batch\n")
train_xml_files = glob.glob('/content/drive/MyDrive/Dataset/train/annotations/*.xml')
# Iterate over each XML file and update class frequencies
for file_path in train_xml_files:
    update_class_frequencies_from_file(file_path, train_total_class_frequencies)

# Print the total occurrences of each class across all images
for class_name, frequency in train_total_class_frequencies.items():
    print(f"{class_name}: freq. {frequency}")


VAL batch



NameError: name 'glob' is not defined

## Calculating weights based off frequencies

Given a dictionary of frequencies, we can determine how much we should weigh certain classes when we train the model

In [ ]:
# GET CLASS WEIGHTS

class_frequencies = {
    'white-bishop': 859,
    'white-pawn': 1242,
    'white-rook': 369,
    'white-knight': 402,
    'white-king': 510,
    'white-queen': 304,
}

# Calculate weights (example using inverse frequency)
total = sum(class_frequencies.values())
class_weights = {k: total / v for k, v in class_frequencies.items()}

# Normalize weights to make the smallest weight 1 (optional)
min_weight = min(class_weights.values())
class_weights_normalized = {k: v / min_weight for k, v in class_weights.items()}

print(class_weights_normalized)


{'white-bishop': 1.4458672875436553, 'white-pawn': 1.0, 'white-rook': 3.3658536585365857, 'white-knight': 3.08955223880597, 'white-king': 2.435294117647059, 'white-queen': 4.0855263157894735}


# Training the  model
Deep learning library, Ultralytics, provides a CLI for custom training/fine-
tuning this object detection model.

"yolo!" command is a shell command that a part of a larger system for running
YOLO tasks. It's not a standard command in common environments but likely
relates to a custom or extended script or toolkit designed to work with YOLO
models. The parameters specify how the model should be trained:

task=detect: Indicates that the task is object detection. YOLO models are
primarily used for detecting objects within images, specifying their
categories and bounding boxes.
mode=train: Sets the mode to training, meaning the model will learn from the
provided dataset rather than performing inference (detection) on new images.
model=yolov8s.pt: Specifies the model file to be used or fine-tuned.
yolov8s.pt suggests a YOLOv8 small variant model, which is a pre-trained model
file. The .pt extension indicates a PyTorch model.
data={dataset.location}/data.yaml: Points to the dataset's configuration file,
which includes paths to the images, labels, and other important information
for training. {dataset.location} is a placeholder that gets replaced with the
actual location of the downloaded dataset.
epochs=25: Specifies the number of training epochs, which is the number of
times the entire dataset will be passed through the network. More epochs can
lead to better learning but also a higher risk of overfitting.
imgsz=800: Sets the image size to 800 pixels for training, which is a
parameter for resizing the input images to maintain consistency.
plots=True: enables the generation of plots or graphs to visualize
aspects of the training process, such as loss metrics or accuracy over time.

In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=25 imgsz=800 plots=True project=/content/drive/MyDrive/saved
# trained second half white images on the existing whites.pt you saved.
#!yolo task=detect mode=train model=/content/drive/MyDrive/saved/train2/weights/whites.pt data={dataset.location}/data.yaml epochs=25 imgsz=800 plots=True project=/content/drive/MyDrive/saved



100% 21.5M/21.5M [00:00<00:00, 169MB/s]
Ultralytics YOLOv8.1.23 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/random-chess-detection-1/data.yaml, epochs=25, time=None, patience=100, batch=16, imgsz=800, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/saved, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt

# Validating the model

In [ ]:
#!yolo task=detect mode=val model=/content/drive/MyDrive/saved/train2/weights/whites.pt data={dataset.location}/data.yaml
!yolo task=detect mode=val model=/content/drive/MyDrive/saved/train/weights/random-white-gray.pt data={dataset.location}/data.yaml


Ultralytics YOLOv8.1.23 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11130615 parameters, 0 gradients, 28.5 GFLOPs
val: Scanning /content/random-chess-detection-1/valid/labels.cache... 635 images, 0 backgrounds, 0 corrupt: 100% 635/635 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 40/40 [00:24<00:00,  1.64it/s]
                   all        635       5153      0.998          1      0.995      0.944
      b-bishoprotation        635        259      0.998          1      0.995      0.941
        b-kingrotation        635        195      0.994          1       0.99      0.933
      b-knightrotation        635        272      0.999          1      0.995      0.957
        b-pawnrotation        635       1190      0.997          1      0.995      0.954
       b-queenrotation        635        169          1      0.999      0.995       0.92
        b-rookrotation        635 

# Running inference

In [ ]:
# RUN INFERENCE
# Update the file path to point to the desired location of the YOLO model .pt file
model_path = '/content/drive/MyDrive/saved/train11/weights/stratified-white-gray.pt'

# Initialize the YOLO model using the updated file path
model = YOLO(model_path)

# Perform object detection using the updated model
#results = model.predict(source='/content/Chess-Pieces-25/test/images')
# halfwhites run: results = model.predict(source='/content/aerial-chess-1/test/images/WIN_20240207_21_29_30_Pro_jpg.rf.9dbad7fb47070c3070e9526e60892e9a.jpg', save=True)
# full white run

#results = model.predict(source='/content/aerial-chess-2/test/images/WIN_20240212_21_26_40_Pro_jpg.rf.bea252199d659ec916b46bb4756e27b7.jpg', save=True)
#results = model.predict(source='/content/aerial-chess-2/test/images/WIN_20240212_21_25_41_Pro_jpg.rf.5765db2f7baea10f7254d7e2bdbd9705.jpg', save=True)
#results = model.predict(source='/content/aerial-chess-2/test/images/WIN_20240212_21_30_16_Pro-2-_jpg.rf.7bdc663c91c1bbd17ab222041507a826.jpg', save=True)
#results = model.predict(source='/content/aerial-chess-2/test/images/WIN_20240212_21_34_25_Pro_jpg.rf.302616ba10ccc21109fabd92cbbb549b.jpg', save=True)

results = model.predict(source='/content/drive/MyDrive/Stratified/Stratified_White_Gray_Dataset/Test/WIN_20240304_19_18_58_Pro.jpg', save=True)


image 1/1 /content/drive/MyDrive/Stratified/Stratified_White_Gray_Dataset/Test/WIN_20240304_19_18_58_Pro.jpg: 480x800 2 b-bishops, 1 b-king, 2 b-knights, 8 b-pawns, 1 b-queen, 2 b-rooks, 1 board, 2 w-bishops, 1 w-king, 2 w-knights, 8 w-pawns, 1 w-queen, 2 w-rooks, 72.8ms
Speed: 19.6ms preprocess, 72.8ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 800)
Results saved to runs/detect/predict5


In [ ]:
# SAVE THE INFERENCE YOLO OUTPUT IMAGES TO GOOGLE DRIVE
# RUN THIS CELL AFTER YOU CREATED A NEW MODEL SO THAT THIS MODEL PERSISTS ACROSS FUTURE SESSIONS OF GOOGLE COLAB. ONLY RUN THIS WHEN MOVING THE NEW RUNS FOLDER.
import shutil

# Specify the paths
source_folder = '/content/runs/detect'  # Replace with the local path of your folder
destination_drive = '/content/drive/MyDrive/saved/inference'   # Replace with your desired Google Drive location

# Move the folder to Google Drive
shutil.move(source_folder, destination_drive)

'/content/drive/MyDrive/saved/inference'

# Extracting YOLO output

Extract the output from custom-trained Ultralytics YOLOv8 output

The following code sets up to produce a text file

In [ ]:
# STORE THE RESULTS
# Extract the output from custom-trained Ultralytics YOLOv8 output

for diane_result in results:
  # Split the string by '/'
  fileNamePath = diane_result.path
  parts = fileNamePath.split('/')
  # Get the last part which contains the file name
  file_name = parts[-1]
  # SLICING
  print ('file name: ', file_name)
  boxes = diane_result.boxes  # Create Box object. Boxes object for bbox outputs
  print ('xyxy coordinates: ', boxes.xyxy)
  print(type(boxes.xyxy))
  print ('confidence rate: ', boxes.conf)
  print ('class name: ', boxes.cls)
  print (boxes.xyxyn)

file name:  WIN_20240304_19_18_58_Pro.jpg
xyxy coordinates:  tensor([[ 518.5357,  105.4212, 1383.4055,  972.0563],
        [ 968.7999,  266.9184, 1027.2419,  334.0478],
        [ 966.2567,  666.5179, 1022.4059,  726.7371],
        [1166.1210,  861.1334, 1218.7379,  924.4923],
        [ 673.9385,  154.0345,  733.0201,  218.2929],
        [1065.7251,  570.7961, 1116.2743,  620.5695],
        [1178.3306,  269.6561, 1233.0710,  324.3505],
        [ 677.4494,  858.7459,  735.4850,  924.5429],
        [ 546.9959,  139.1569,  632.0518,  221.3523],
        [ 758.2723,  151.3987,  830.9456,  227.3768],
        [1066.4695,  153.0081, 1138.0762,  228.7300],
        [1265.8591,  660.0791, 1339.5753,  722.0930],
        [1273.5112,  149.3601, 1351.4485,  227.1409],
        [ 672.7525,  668.6711,  731.1196,  722.5224],
        [ 547.8330,  361.3055,  627.6331,  424.1643],
        [ 871.3150,  473.6402,  920.7028,  523.6343],
        [ 553.3124,  850.3185,  638.8500,  934.0967],
        [1178.4885,  

The following code down below actually produces the text file

In [ ]:
# Write the extracted YOLO output to a text file to send to Diane

import numpy as np

# Define the class_mapping dictionary
class_mapping = {
    0: 'b-bishop',
    1: 'b-king',
    2: 'b-knight',
    3: 'b-pawn',
    4: 'b-queen',
    5: 'b-rook',
    6: 'board',
    7: 'w-bishop',
    8: 'w-king',
    9: 'w-knight',
   10: 'w-pawn',
   11: 'w-queen',
   12: 'w-rook'
}

# Open a text file for writing
with open('4_gray_white.txt', 'w') as file:
    # Write the file name to the text file
    file.write('file name:\n')
    file.write(file_name + '\n')  # Write the file name and add a newline character

    # Write the xy coordinates to the text file
    file.write('xyxy coordinates:\n')
    np.savetxt(file, boxes.xyxy.cpu().numpy(), fmt='%f')
    file.write('\n')

    # Write the confidence rates to the text file
    file.write('confidence rate:\n')
    np.savetxt(file, boxes.conf.cpu().numpy(), fmt='%f')
    file.write('\n')

    # Write the class names and their respective numbers to the text file
    file.write('class number - class name:\n')
    class_indices = boxes.cls.cpu().numpy().astype(int)  # Convert class indices to integers
    for class_index in class_indices:
        class_name = class_mapping[class_index]
        file.write(f'{class_index} - {class_name}\n')

The following code down below saves a file/folder to another folder

In [ ]:
# SAVE THE DIANE TEXTFILE WE WROTE TO
# RUN THIS CELL AFTER YOU CREATED A NEW MODEL SO THAT THIS MODEL PERSISTS ACROSS FUTURE SESSIONS OF GOOGLE COLAB. ONLY RUN THIS WHEN MOVING THE NEW RUNS FOLDER.
# import shutil

# # Specify the paths
# source_folder = ''  # Replace with the local path of your folder
# destination_drive = ''   # Replace with your desired Google Drive location

# # Move the folder to Google Drive
# shutil.move(source_folder, destination_drive)